In [ ]:
import pandas as pd
df_sample = pd.read_csv("/Users/htg/Desktop/Sydney_University/冒犯性检测/dataset/data_emnlp.csv", sep=",")


In [ ]:
#QWQ-32B
import requests
import pandas as pd
import time
import re  

url = "https://api.siliconflow.cn/v1/chat/completions"

# 读取数据集

texts = df_sample.iloc[:, 0]

headers = {
    "Authorization": "Bearer Your own key",  
    "Content-Type": "application/json"
}

results = []

# 逐条处理文本
for idx, text in enumerate(texts):
    payload = {
        "model": "Qwen/QwQ-32B",
        "messages": [
            {
                "role": "user",
                "content": "你是一个中文冒犯性语言检测专家。\n"
                            "请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容\n"
                            "这里的冒犯性主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。\n" 
                            "如果文本具有冒犯性，请仅回复数字 1；如果不具有冒犯性，请仅回复数字 0。\n"
                            f"文本如下：\n{text}\n"
            }
        ],
        "stream": False,
        "max_tokens": 10000,
        "stop": None,
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 5,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"},
       
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    # 检查 API 响应
    if response.status_code == 200:
        json_response = response.json()
        if "choices" in json_response and len(json_response["choices"]) > 0:
            result = json_response["choices"][0]["message"]["content"]
            # 提取 "</think>" 后面的数字
            result_stripped = result.strip()
            match = re.search(r"</think>\s*(\d+)", result_stripped)
            if match:
                extracted = match.group(1)
            else:
                extracted = result_stripped
            print(f"第 {idx+1} 条数据判断结果：{extracted}")
            results.append(extracted)

    time.sleep(2)  

# 保存结果到 CSV 文件
result_df = pd.DataFrame({"Content": texts, "Result": results})

result_df.to_csv("./base_result/QWQ_results.csv", sep="\t", index=False)